In [2]:
import pygame, random, sys, os, time
from pygame.locals import *

# Настройки 
WINDOWWIDTH = 800
WINDOWHEIGHT = 600
TEXTCOLOR = (255, 255, 255)
BACKGROUNDCOLOR = (0, 0, 0)
FPS = 40
BADDIEMINSPEED = 8
BADDIEMAXSPEED = 8
ADDNEWBADDIERATE = 6
PLAYERMOVERATE = 5

# Завершение игры 
def terminate():
    pygame.quit()
    sys.exit()  

# Ожидание нажатия клавиши 
def waitForPlayerToPressKey():
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    terminate()
                return

# Проверка столкновения с машиной
def playerHasHitBaddie(playerRect, baddies):
    for b in baddies:
        if playerRect.colliderect(b['rect']):
            return True
    return False  

# Проверка столкновения с монетой
def playerHasHitCoin(playerRect, coins):
    for c in coins:
        if playerRect.colliderect(c['rect']):
            coins.remove(c)
            return True
    return False

# Отрисовка текста
def drawText(text, font, surface, x, y, center=False):
    textobj = font.render(text, True, TEXTCOLOR)
    textrect = textobj.get_rect()
    if center:
        textrect.center = (x, y)
    else:
        textrect.topleft = (x, y)
    surface.blit(textobj, textrect) 

# Инициализация 
pygame.init()
mainClock = pygame.time.Clock()
windowSurface = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
pygame.display.set_caption('Racer')
pygame.mouse.set_visible(False)

# Шрифты 
font = pygame.font.SysFont(None, 30)

# Звуки 
try:
    gameOverSound = pygame.mixer.Sound('racer_files/crash.wav')  # Проверяем звук при аварии
    pygame.mixer.music.load('racer_files/car.wav')  # Фоновая музыка
    laugh = pygame.mixer.Sound('racer_files/laugh.wav')  # Звук смеха
    print("Звуки успешно загружены!")
except Exception as e:
    print(f"Ошибка загрузки звуков: {e}")

# Изображения
playerImage = pygame.image.load('racer_files/car1.png')
car3 = pygame.image.load('racer_files/car3.png')
car4 = pygame.image.load('racer_files/car4.png')
coinImage = pygame.image.load('racer_files/Coin.png')
baddieImage = pygame.image.load('racer_files/car2.png')
wallLeft = pygame.image.load('racer_files/left.png')
wallRight = pygame.image.load('racer_files/right.png')

sampleCars = [car3, car4, baddieImage]
playerRect = playerImage.get_rect()

# Начальный экран 
drawText('Press any key to start the game.', font, windowSurface, WINDOWWIDTH // 2, 200, center=True)
drawText('And Enjoy', font, windowSurface, WINDOWWIDTH // 2, 240, center=True)
pygame.display.update()
waitForPlayerToPressKey()  

# Чтение топового счёта 
if not os.path.exists("data"):
    os.makedirs("data")
if not os.path.exists("data/save.dat"):
    with open("data/save.dat", 'w') as f:
        f.write("0")
with open("data/save.dat", 'r') as f:
    topScore = int(f.readline())

# Главный цикл
life = 3
while life > 0:
    baddies = []
    coins = []
    score = 0
    coinsCollected = 0
    playerRect.topleft = (WINDOWWIDTH // 2 - 25, WINDOWHEIGHT - 100)  # Центрируем машину
    moveLeft = moveRight = moveUp = moveDown = False
    baddieAddCounter = 0
    pygame.mixer.music.play(-1, 0.0)  
    
    while True:
        score += 1
        for event in pygame.event.get():
            if event.type == QUIT:
                terminate()
            if event.type == KEYDOWN:
                if event.key in (K_LEFT, ord('a')):
                    moveLeft = True
                if event.key in (K_RIGHT, ord('d')):
                    moveRight = True
                if event.key in (K_UP, ord('w')):
                    moveUp = True
                if event.key in (K_DOWN, ord('s')):
                    moveDown = True
            if event.type == KEYUP:
                if event.key == K_ESCAPE:
                    terminate()
                if event.key in (K_LEFT, ord('a')):
                    moveLeft = False
                if event.key in (K_RIGHT, ord('d')):
                    moveRight = False
                if event.key in (K_UP, ord('w')):
                    moveUp = False
                if event.key in (K_DOWN, ord('s')):
                    moveDown = False   
        
        baddieAddCounter += 1
        if baddieAddCounter >= ADDNEWBADDIERATE:
            baddieAddCounter = 0
            baddieSize = 30
            # Машины появляются только в пределах дороги (между 140 и 485)
            newBaddie = {
                'rect': pygame.Rect(random.randint(140, 485), -40, 23, 47),  # Позиция машин между стенами
                'speed': random.randint(BADDIEMINSPEED, BADDIEMAXSPEED),
                'surface': pygame.transform.scale(random.choice(sampleCars), (23, 47))
            }
            baddies.append(newBaddie) 
            # Создание монеты в свободной области
            for _ in range(5):
                coin_x = random.randint(140, 485)
                coin_rect = pygame.Rect(coin_x, -30, 23, 23)
                overlap = False
                for b in baddies:
                    if coin_rect.colliderect(b['rect']):
                        overlap = True
                        break
                if not overlap:
                    newCoin = {
                        'rect': coin_rect,
                        'speed': newBaddie['speed'],
                        'surface': pygame.transform.scale(coinImage, (23, 23))
                    }
                    coins.append(newCoin)
                    break 
        
        # Движение игрока, ограничиваем движение внутри зоны
        if moveLeft and playerRect.left > 140: 
            playerRect.move_ip(-PLAYERMOVERATE, 0)
        if moveRight and playerRect.right < 485:
            playerRect.move_ip(PLAYERMOVERATE, 0)
        if moveUp and playerRect.top > 0:
            playerRect.move_ip(0, -PLAYERMOVERATE)
        if moveDown and playerRect.bottom < WINDOWHEIGHT:
            playerRect.move_ip(0, PLAYERMOVERATE)

        # Обновление объектов
        for b in baddies[:]:
            b['rect'].move_ip(0, b['speed'])
            if b['rect'].top > WINDOWHEIGHT:
                baddies.remove(b) 
        for c in coins[:]:
            c['rect'].move_ip(0, c['speed'])
            if c['rect'].top > WINDOWHEIGHT:
                coins.remove(c)   

        windowSurface.fill(BACKGROUNDCOLOR)

        # Отображение дороги (стены по бокам)
        windowSurface.blit(wallLeft, (40, 0))  # Левый край дороги
        windowSurface.blit(wallRight, (500, 0))  # Правый край дороги

        # Информация
        drawText(f'Score: {score}', font, windowSurface, 10, 0)
        drawText(f'Top Score: {topScore}', font, windowSurface, 10, 20)
        drawText(f'Life: {life}', font, windowSurface, 10, 40)
        drawText(f'Coins: {coinsCollected}', font, windowSurface, 10, 60)

        windowSurface.blit(playerImage, playerRect)

        # Отображение машин и монет
        for b in baddies:
            windowSurface.blit(b['surface'], b['rect'])
        for c in coins:
            windowSurface.blit(c['surface'], c['rect'])

        pygame.display.update()  
        
        # Проверка на столкновения
        if playerHasHitBaddie(playerRect, baddies):
            if score > topScore:
                with open("data/save.dat", 'w') as f:
                    f.write(str(score))
                topScore = score
            # Воспроизведение звука аварии
            try:
                gameOverSound.play()
            except:
                print("Ошибка воспроизведения звука аварии.")
            break

        if playerHasHitCoin(playerRect, coins):
            coinsCollected += 1

        mainClock.tick(FPS)    

    pygame.mixer.music.stop()
    life -= 1
    if life == 0:
        try:
            laugh.play()  # Звук смеха при проигрыше
        except:
            print("Ошибка воспроизведения смеха.")
        drawText('Game Over', font, windowSurface, 300, 250)
        drawText('Press any key to restart.', font, windowSurface, 240, 290)
        pygame.display.update()
        time.sleep(2)
        waitForPlayerToPressKey()
        life = 3


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Звуки успешно загружены!


SystemExit: 

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
